# __Gesamtdatenanalyse__

In diesem Teil werden die Stelleninserate untersucht, indem die Wörter aus dem Wörterbuch im Inserat gezählt werden.

In [1]:
# Import der benötigten Libraries
import pandas as pd
import nltk
import numpy as np
import re
import os

pd.set_option('display.max_colwidth', 50)

# Working-Directory
os.chdir('C:/Users/Sven Konermann/Documents/Master/2_Semester/aF_E_Faelle/Tuning/04_IT_Projektmanagement')
print(os.getcwd())

# Files im Directory
print(os.listdir())

C:\Users\Sven Konermann\Documents\Master\2_Semester\aF_E_Faelle\Tuning\04_IT_Projektmanagement
['Anzahl_Inserate_Anforderungsdaten.pdf', 'Anzahl_Inserate_Gesamtdaten.pdf', 'Inserate_Anforderungsanalyse.csv', 'Inserate_clean_Anforderungsanalyse.csv', 'Inserate_clean_Gesamtdatenanalyse.csv', 'Inserate_Gesamtdatenanalyse.csv', 'ngram_count_all_Data.csv', 'Subset_IT_Projektmanagement.csv', 'word_count_all_Data.csv']


In [2]:
data = pd.read_csv("Inserate_clean_Gesamtdatenanalyse.csv", sep=";")
data = data[~data['Text_Anforderungen'].isnull()] # Wenn NULL-Werte, lösche diese
print(data.shape)

(5343, 11)


Suche nach Teil-Wörtern und Phrasen

In [4]:
# Einlesen vordefiniertes Wörterbuch
set_phrases = pd.read_excel('Phrasenwoerter.xlsx')
set_phrases.head()

,word,kategorie,kategorie_name,hauptkategorie
0,visual studio,2,Informatik Software-Kenntnisse,IT-Kenntnisse
1,big data,3,Datenmanagement,IT-Kenntnisse
2,business intelligence,3,Datenmanagement,IT-Kenntnisse
3,data science,3,Datenmanagement,IT-Kenntnisse
4,datenbank,3,Datenmanagement,IT-Kenntnisse


In [5]:
# Vergleichen unique words, ob sie vorkommen im Text_new. Unique words list kommt vom Tokanizing oben.
year = data['year'].min()
count = 0
while year <= data['year'].max():
    wordsets_phrases = data.loc[data['year'] == year]['Text_Anforderungen'] # Text Detok, da n-grams nach dem gesucht wurden
    for word in set_phrases['word']:
        count = sum(1 for s in wordsets_phrases if word in s)
        index = set_phrases[set_phrases['word']==word].index
        set_phrases.loc[index, year] = count
        set_phrases.loc[index, str(year) + ' %'] = (count / len(wordsets_phrases))    
    
    year += 1

Suche nach Fix-Wörtern, die genau so vorkommen müssen

In [7]:
set_fix = pd.read_excel("Fixwoerter.xlsx")

In [8]:
# Anforderungen Tokenisieren
set_fix['text_tok'] = set_fix.apply(lambda row: nltk.word_tokenize(row['word']), axis=1)
set_fix['text_tok'].head(10)

0      [abap]
1         [c]
2      [java]
3     [plsql]
4    [python]
5         [r]
6    [apache]
7     [azure]
8    [citrix]
9      [hana]
Name: text_tok, dtype: object

In [9]:
# Anforderungen in einem Set speichern
new_set = set()
set_fix['text_tok'].apply(new_set.update)
u = new_set

In [10]:
# Vergleiche unique words, ob sie vorkommen im Text_new. Unique words list kommt vom Tokanizing oben.
year = data['year'].min()

while year <= data['year'].max():    
    wordsets_fix = [ frozenset(document.split(' ')) for document in data.loc[data['year'] == year]['Text_Anforderungen'] ]
    for word in u:
        count = sum( 1 for s in wordsets_fix if word in s )
        index = set_fix[set_fix['word']==word].index
        set_fix.loc[index, year] = count
        set_fix.loc[index, str(year) + ' %'] = (count / len(wordsets_phrases))

    year += 1

In [11]:
anforderungen = set_phrases.append(set_fix, ignore_index=True)

C:\Users\Sven Konermann\AppData\Local\Temp\ipykernel_61460\2611590460.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  anforderungen = set_phrases.append(set_fix, ignore_index=True)


In [12]:
# Anforderungen mit den meisten Nennungen
anforderungen.sort_values(by=2021,ascending=False).head(5)

,word,kategorie,kategorie_name,hauptkategorie,2014,2014 %,2015,2015 %,2016,2016 %,...,2017 %,2018,2018 %,2019,2019 %,2020,2020 %,2021,2021 %,text_tok
10,deutsch,8,Sprachkenntnisse,Sprachen,501.0,0.673387,412.0,0.642746,445.0,0.645864,...,0.656552,511.0,0.687752,465.0,0.665236,441.0,0.732558,334.0,0.668,NaN
11,englisch,8,Sprachkenntnisse,Sprachen,406.0,0.545699,311.0,0.485179,353.0,0.512337,...,0.544828,413.0,0.555855,378.0,0.540773,332.0,0.551495,256.0,0.512,NaN
21,teamfähig,12,Soft Skills,Soft Skills,296.0,0.397849,208.0,0.324493,232.0,0.336720,...,0.348966,299.0,0.402423,274.0,0.391989,247.0,0.410299,202.0,0.404,NaN
25,selbständig,12,Soft Skills,Soft Skills,179.0,0.240591,164.0,0.255850,163.0,0.236575,...,0.251034,193.0,0.259758,211.0,0.301860,175.0,0.290698,131.0,0.262,NaN
68,berufserfahrung,9,Berufserfahrung,Abschlüsse und Berufserfahrung,148.0,0.296000,116.0,0.232000,146.0,0.292000,...,0.422000,171.0,0.342000,174.0,0.348000,145.0,0.290000,101.0,0.202,[berufserfahrung]


In [13]:
# Nur für Daten mit Jobcloud
anforderungen.to_csv("ausgewertete_Anforderungen.csv", sep=";")